In [1]:
!pip install pyspark >> None

# Задача 1

У Вас есть RDD со следующей структуройЖ (ключ, значение). Напишите программу на Apache Spark, которая найдет среднее значение для каждого уникального ключа.

In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local[*]").setAppName("Average By Key Example")
sc = SparkContext(conf = conf)

dataRDD = sc.parallelize([
    ("key1", 10),
    ("key2", 20),
    ("key1", 30),
    ("key2", 40),
    ("key1", 50)
])

avarageRDD = dataRDD \
   .mapValues(lambda value: (value, 1)) \
   .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
   .mapValues(lambda sum_count: sum_count[0] / sum_count[1])

# Collect the data and print it
result = avarageRDD.collect()

for key, value in result:
    print(f"{key}: {value}")

sc.stop()

key1: 30.0
key2: 30.0


# Задача 2

У Вас есть RDD со следующей структуройЖ (ключ, значение). Напишите программу на Apache Spark, которая найдет максимальное значение для каждого уникального ключа.

In [3]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local[*]").setAppName("MaxValueByKeyExample")
sc = SparkContext(conf = conf)

data = ([
    ("key1", 10),
    ("key2", 5),
    ("key1", 20),
    ("key2", 15),
    ("key3", 8)
])

rdd = sc.parallelize(data)

max_values = rdd.reduceByKey(max)

results = max_values.collect()
for result in results:
    print(f"{result[0]}: {result[1]}")

sc.stop()

key1: 20
key2: 15
key3: 8


# Задача 3

Напишите программу на Apache Spark, которая считает сумму всех чисел в заданном RDD.

In [4]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local[*]").setAppName("SumRDDExample")
sc = SparkContext(conf = conf)

data = [1, 2, 3, 4, 5]

rdd = sc.parallelize(data)

total_sum = rdd.sum()

print("Total_sum =", total_sum)

sc.stop()

Total_sum = 15


# Задача 4

У Вас есть RDD со следующей структурой: (ключ, значение). Напишите программу на Apache Spark, которая найдет топ N ключей с наибольшим значением, где N - заданное число.

In [5]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local[*]").setAppName("TopKeysExample")
sc = SparkContext(conf = conf)

data = [
    ("key1", 10),
    ("key2", 5),
    ("key1", 20),
    ("key2", 15),
    ("key3", 8)
]

rdd = sc.parallelize(data)

N = 3

top_N_keys = rdd.takeOrdered(N, key=lambda x: -x[1])

for key, value in top_N_keys:
    print(f"{key}: {value}")

sc.stop()

key1: 20
key2: 15
key1: 10


# Задача 5

Напишите программу на Apache Spark, которая читает CSV-файл и выводит количество строк в файле.

In [6]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setMaster("local[*]").setAppName("CountCSVLinesExample")
sc = SparkContext(conf = conf)

spark = SparkSession.builder.getOrCreate()

# Create data and write it to csv
data = [
    ("key1", 10),
    ("key2", 5),
    ("key1", 20),
    ("key2", 15),
    ("key3", 8)
]
columns = ["key", "value"]
df = spark.createDataFrame(data, columns)
df.write.csv("./data.csv", header=True)

# Get data from csv
csv_file_path = "./data.csv"
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

line_count = df.count()

print("Line count =", line_count)

sc.stop()

Line count = 5


# Задача 6

У Вас есть RDD со следующей структурой: (ключ, значение). Напишите программу на Apache Spark, которая найдет сумму значений для каждого уникального ключа, но только для ключей, значение которых больше заданного порога.

In [7]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local[*]").setAppName("SumValuesAboveThresholdExample")
sc = SparkContext(conf = conf)

data = [
    ("key1", 10),
    ("key2", 20),
    ("key1", 30),
    ("key3", 40),
    ("key2", 50)
]

rdd = sc.parallelize(data)

threshold = 25

filtered_rdd = rdd.filter(lambda x: x[1] > threshold)

result = filtered_rdd.reduceByKey(lambda a, b: a + b)

for key, value in result.collect():
    print(f"{key}: {value}")

sc.stop()

key1: 30
key2: 50
key3: 40


# Задача 7

У Вас есть RDD со следующей структурой: (ключ, список значений). Напишите программу на Apache Spark, которая найдет среднюю длину списка для каждого ключа.

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("AverageListLengthExample").getOrCreate()

data = [
    ("key1", [1, 2, 3]),
    ("key2", [4, 5, 6]),
    ("key1", [7, 8, 9, 4, 7]),
    ("key2", [10, 11, 12, 9, 4, 7]),
    ("key3", [13, 14, 15])
]

rdd = spark.sparkContext.parallelize(data)

rdd_key_length = rdd.mapValues(lambda x: len(x))

rdd_sum_count = rdd_key_length.combineByKey(
    lambda x: (x, 1),
    lambda acc, x: (acc[0] + x, acc[1] + 1),
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])
)

rdd_avg_length = rdd_sum_count.mapValues(lambda x: x[0] / x[1])

result = rdd_avg_length.collect()

for key, avg_length in result:
    print(f"{key}: {avg_length}")

spark.stop()

key1: 4.0
key2: 4.5
key3: 3.0


# Задача 8

У Вас есть RDD, содержащий записи о посещениях веб-сайта со следующей структурой: (пользователь, дата, время). Напишите программу на Apache Spark, которая найдет количество уникальных пользователей для каждой даты.

In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("UniqueUsersPerDataExample").getOrCreate()

data = [
    ("user1", "2024-04-01", "10:00:00"),
    ("user2", "2024-04-01", "11:00:00"),
    ("user3", "2024-04-01", "12:00:00"),
    ("user1", "2024-04-02", "13:00:00"),
    ("user2", "2024-04-02", "14:00:00"),
    ("user3", "2024-04-02", "15:00:00"),
    ("user1", "2024-04-03", "16:00:00"),
]

rdd = spark.sparkContext.parallelize(data)

rdd_data_user = rdd.map(lambda x: (x[1], x[0]))

rdd_unique_users = rdd_data_user.distinct()

result = rdd_unique_users.countByKey()

for date, count in result.items():
    print(f"{date}: {count}")

spark.stop()

2024-04-01: 3
2024-04-02: 3
2024-04-03: 1


# Задача 9

У Вас есть два RDD с одинаковой структурой: (ключ, значение). Напишите программу на Apache Spark, которая выполнит объединение (join) этих RDD по ключу и найдет сумму значений для каждого уникального ключа.

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RDDJoinAndSumExample").getOrCreate()

data1 = [
    ("key1", 10),
    ("key2", 20),
    ("key3", 30),
]

rdd1 = spark.sparkContext.parallelize(data1)

data2 = [
    ("key1", 40),
    ("key2", 50),
    ("key4", 60),
]

rdd2 = spark.sparkContext.parallelize(data2)

rdd_joined = rdd1.join(rdd2)

rdd_sum = rdd_joined.mapValues(lambda x: x[0] + x[1])

result = rdd_sum.collect()

for key, value in result:
    print(f"{key}: {value}")

spark.stop()

key1: 50
key2: 70


# Задача 10

Подсчет количества уникальных слов в тексте с использованием DataFrame.

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split
from pyspark.sql.types import StringType

spark = SparkSession.builder \
    .appName("CountUniqueWordsExample") \
    .master("local[*]") \
    .getOrCreate()

text = [
    "Hello, Spark",
    "Hello, World",
    "Hello, Scala"
    ]

textDF = spark.createDataFrame(text, StringType()).toDF("text")
wordsDF = textDF.select(explode(split("text", " ")).alias("word"))

result = wordsDF.distinct().count()
print(result)

spark.stop()

4


# Задача 11

Объектно-ориентированный подход с использованием DataFrame API.

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("OOPExample") \
    .getOrCreate()

data = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 30},
    {"name": "Charlie", "age": 35}
]

df = spark.createDataFrame(data)

df_filtered = df.filter(df.age > 30)
df_filtered.show()

spark.stop()

+---+-------+
|age|   name|
+---+-------+
| 35|Charlie|
+---+-------+

